# Introduction

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('datasets/recipes/LibriGenerate_eval.csv')
df.head(5)

,clean_utterance_path,embedding_utterance_path,interference_utterance_path,clean_segment_start,interference_segment_start,segment_length,loss_Re_eval_PSE_DCCRN,sdr_before_Re_eval_PSE_DCCRN,sdr_after_Re_eval_PSE_DCCRN,loss_Test_vf,sdr_before_Test_vf,sdr_after_Test_vf,loss_Re_eval_PSE_DCCRN_big,sdr_before_Re_eval_PSE_DCCRN_big,sdr_after_Re_eval_PSE_DCCRN_big
0,datasets/LibriSpeech/test-clean/2094/142345/20...,datasets/LibriSpeech/test-clean/2094/142345/20...,datasets/LibriSpeech/dev-clean/8842/304647/884...,499,84191,3.0,0.023860,3.115668,5.993759,0.017454,3.115668,9.964132,0.018467,3.115668,8.171122
1,datasets/LibriSpeech/test-clean/8455/210777/84...,datasets/LibriSpeech/test-clean/8455/210777/84...,datasets/LibriSpeech/test-clean/2300/131720/23...,37312,34157,3.0,0.040819,-1.555514,-0.106652,0.026095,-1.555514,3.483958,0.025756,-1.555514,4.777966
2,datasets/LibriSpeech/test-clean/4992/23283/499...,datasets/LibriSpeech/test-clean/4992/23283/499...,datasets/LibriSpeech/dev-clean/2412/153947/241...,4600,23807,3.0,0.026882,3.569494,5.295632,0.019188,3.569494,11.549106,0.020897,3.569494,7.999767
3,datasets/LibriSpeech/dev-clean/777/126732/777-...,datasets/LibriSpeech/dev-clean/777/126732/777-...,datasets/LibriSpeech/dev-clean/2803/154320/280...,95090,8409,3.0,0.023600,-7.042596,0.438827,0.022958,-7.042596,2.107655,0.026215,-7.042596,-1.908049
4,datasets/LibriSpeech/test-clean/4992/41806/499...,datasets/LibriSpeech/test-clean/4992/41806/499...,datasets/LibriSpeech/dev-clean/3853/163249/385...,56795,111033,3.0,0.032140,-9.065487,-4.019209,0.053245,-9.065487,-6.167892,0.031171,-9.065487,-3.058802


# Main app

In [3]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px

app = JupyterDash('Explore test result')

In [25]:
app.layout = html.Div([
    
    html.Div(children=[
        html.H2("Dimensions"),

        html.Div(children=[
            html.Div(children=[
                html.Label('X-axis'),
                dcc.Dropdown(df.columns, None, id="dim-x-axis"),
            ]),

            html.Div(children=[
                html.Br(),
                html.Label('Y-axis'),
                dcc.Dropdown(df.columns, None, id="dim-y-axis"),
            ]),
        ]),

        html.Br(),
        html.Label('Color'),
        dcc.Dropdown(df.columns, None, id="dim-color"),

        html.Br(),
        html.Label('Size'),
        dcc.Dropdown(df.columns, None, id="dim-size"),
    ], style={'padding': 10, 'flex': 1}),

    html.Div(children=[
        html.H2("Interactive plot"),
        dcc.Graph(id='scatter-plot')
    ], style={'padding': 10, 'flex': 3})
], style={'display': 'flex', 'flex-direction': 'row'})

In [5]:
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('dim-x-axis', 'value'),
    Input('dim-y-axis', 'value'),
    Input('dim-color', 'value'),
    Input('dim-size', 'value'),
)
def update_plot_dims(x_axis, y_axis, color, size):
    fig = px.scatter(df, x=x_axis, y=y_axis,
                 size=size, color=color, hover_name=None,
                 log_x=False, size_max=55)
    fig.update_layout(transition_duration=500)
    return fig

In [6]:
# Run app and display result inline in the notebook
# app.run_server(mode='inline')
app.run_server()

Dash app running on http://127.0.0.1:8050/
